In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import time
import re
import os
import pandas as pd
entries = os.listdir("/volumes/Hayley's Drive/PycharmProjects/twilightvalefalls/")
dir = "/volumes/Hayley's Drive/PycharmProjects/twilightvalefalls/"

In [2]:
def countWords(string): 
    OUT = False
    IN = True
    state = OUT 
    wc = 0
  
    # Scan all characters one by one 
    for i in range(len(string)): 
  
        # If next character is a separator,  
        # set the state as OUT 
        if (string[i] == ' ' or string[i] == '\n' or
            string[i] == '\t'): 
            state = OUT 
  
        # If next character is not a word  
        # separator and state is OUT, then  
        # set the state as IN and increment  
        # word count 
        elif state == OUT: 
            state = IN 
            wc += 1
            
    return wc

In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
cleaned_name_entity_list = list()

In [5]:
s = simple_get('https://gravityfalls.fandom.com/wiki/Category:Characters')
b = BeautifulSoup(s,'html.parser')
b = b.find_all('div', attrs={'class',"category-page__members"})
char_links = re.findall(r'href=\"[\w\/\%\.\(\)]*\"', str(b))
char_links = [re.sub(r'(href=\")([\w\/\%\.\(\)]*)(\")',r'\2',link) for link in char_links]

In [6]:
nicknames_dict = dict()

for link in char_links:
    s = simple_get('https://gravityfalls.fandom.com' + link)
    s = BeautifulSoup(s,'html.parser')
    div = s.find_all('div', attrs={'data-source':'alias'})
    
    if not div is None:
        add_on = re.sub(r'(\/wiki\/)([\w\%]*)',r'\2',link)
        add_on = add_on.replace("_"," ").replace('%27',"'").replace('%C3%A9','é')
        div = re.sub(r'[<div\sclass=\"[\w\s\-]*\"\sdata\-source="alias">\s<h3\sclass=\"[\w\s\-]*\">Other names<\/h3>\s','',str(div))
        start = div.find('>')
        current = start
        allpieces = list()
        nicknames = list()
        nick = list()
        while(current != len(div)-1):
            if div[current] == '<':
                allpieces.append(div[start:current])
                start = current
            current += 1

        for p in allpieces:
            #print('piece: ', p.strip())
            if not 'cite' in p and not 'v>' in p and not 'i>' in p and len(p.strip())>4 and not '</sup>' in p and p != '<br/>' and not 'class' in p and not 'prison' in p.lower():
                #print('appended: ', p.strip())
                nicknames.append(p)
        
        for n in nicknames:
            fin_tag = ''
            start = n.find('>')
            parenS = n.find('(')
            parenE = n.find(')')
            
            #print(n)
            if parenS != -1:
                paren_word = n[parenS+1:parenE]
                #print('paren_word:', paren_word)
                if 'and' in add_on:
                    split_add_on = add_on.split(' and ')
                    if countWords(add_on.strip()) == 4:
                        second_str = split_add_on[1]
                        #print(second_str)
                        for i in range(len(second_str)-1,0,-1):
                            #print(i)
                            if second_str[i] == ' ':
                                break
                        extra_bit = second_str[i:]
                        #print(split_add_on[0])
                        first_str = split_add_on[0]
                        del split_add_on[0]
                        split_add_on.insert(0,first_str+extra_bit)
                        #print(split_add_on[0])
                        
                    for s in split_add_on:
                        if countWords(s.strip()) >= 2:
                            fin_tag = ':' + re.sub(r'([\w]{1})([\w\s\']*)\s([\w]*)',r'\1\3', s)
                        #print(s, n[start+1:parenS-1])
                        
                        if paren_word in s:
                            #print('true')
                            if countWords(s.strip()) == 1:
                                #fin_tag = ':' + re.sub(r'([\w]{3})([\w\s]*)',r'\1',s).upper()
                                fin_tag = ':' + s.strip()
                            nick.append(n[start+1:parenS-1] + '::CHR' + fin_tag)
                else:
                    if countWords(add_on.strip()) >= 2:
                            fin_tag = ':' + re.sub(r'([\w]{1})([\w\s\']*)\s([\w]*)',r'\1\3', add_on)
                    elif countWords(add_on.strip()) == 1:
                        #fin_tag = ':' + re.sub(r'([\w]{3})([\w\s]*)',r'\1',add_on).upper()
                        fin_tag = ':' + add_on.strip()
                    #print(add_on, n[start+1:parenS-1])
                    nick.append(n[start+1:parenS-1] + '::CHR' + fin_tag)
            elif parenS == -1:
                if 'and' in add_on:
                    for s in add_on.split(' and '):
                        if countWords(s.strip()) >= 2:
                            fin_tag = ':' + re.sub(r'([\w]{1})([\w\s\']*)\s([\w]*)',r'\1\3', s)
                        elif countWords(s.strip()) == 1:
                            #fin_tag = ':' + re.sub(r'([\w]{3})([\w\s]*)',r'\1',s).upper()
                            fin_tag = ':' + add_on.strip()
                        #print(s, n[start+1:])
                        nick.append(n[start+1:] + '::CHR' + fin_tag)
                        
                else:
                    if countWords(add_on.strip()) >= 2:
                        fin_tag = ':' + re.sub(r'([\w]{1})([\w\s\']*)\s([\w]*)',r'\1\3', add_on)
                    elif countWords(add_on.strip()) == 1:
                        #fin_tag = ':' + re.sub(r'([\w]{3})([\w\s]*)',r'\1',add_on).upper()
                        fin_tag = ':' + add_on.strip()
                    #print(add_on, n[start+1:])
                    nick.append(n[start+1:] + '::CHR' + fin_tag) 
        
        if nicknames:
            nicknames_dict[link] = nick
        else:
            nicknames_dict[link] = add_on
            
        #print(nick)
            
nicknames_set = set()            
for entry in nicknames_dict:
    
    fin_tag = ''
    
    original_name = re.sub(r'(\/wiki\/)([\w\%\.]*)',r'\2',entry)
    original_name = original_name.replace('_',' ').replace('%27',"'").replace('%C3%A9','é')
    
    if ' and ' in original_name:
        split_add_on = original_name.split(' and ')
        
        if countWords(original_name.strip()) == 4:
            second_str = split_add_on[1]
            #print(second_str)
            for i in range(len(second_str)-1,0,-1):
                #print(i)
                if second_str[i] == ' ':
                    break
            extra_bit = second_str[i:]
            #print(split_add_on[0])
            first_str = split_add_on[0]
            del split_add_on[0]
            split_add_on.insert(0,first_str+extra_bit)
            #print(split_add_on[0])
            
        for s in split_add_on:
            
            if countWords(s.strip()) == 1:
                #fin_tag = ':' + re.sub(r'([\w]{3})([\w\s]*)',r'\1',s).upper()
                fin_tag = ':' + s.strip()
            elif countWords(s.strip()) == 2:
                fin_tag = ':' + re.sub(r'([\w]{1})([\w\']*)\s([\w]*)',r'\1\3', s)
            
            #print(s + '::CHR' + fin_tag)   
            nicknames_set.add(s + '::CHR' + fin_tag)
        
        
    else: 
        if countWords(original_name.strip()) == 1:
                #fin_tag = ':' + re.sub(r'([\w]{3})([\w\s]*)',r'\1',original_name).upper()
                fin_tag = ':' + original_name.strip()
        elif countWords(original_name.strip()) >= 2:
            fin_tag = ':' + re.sub(r'([\w]{1})([\w\s\']*)\s([\w]*)',r'\1\3', original_name)
        original_name = original_name + '::CHR' + fin_tag
        #print(original_name)
        nicknames_set.add(original_name)
        
        n = nicknames_dict.get(entry)
    
    if not type(n) is str:
        for i in n:
            i = i.replace('/&gt;','')
            nicknames_set.add(i)
        
for s in nicknames_set:
    cleaned_name_entity_list.append(s)

In [7]:
s = simple_get('https://gravityfalls.fandom.com/wiki/Category:Creatures')
b = BeautifulSoup(s,'html.parser')
b = b.find_all('div', attrs={'class',"category-page__members"})
cre_links = re.findall(r'href=\"[\w\/\%\.\(\)\-]*\"', str(b))

cre_links = [re.sub(r'(href=\")([\w\/\%\.\(\)\-]*)(\")',r'\2',link) for link in cre_links]

for link in cre_links:
    original_name = re.sub(r'(\/wiki\/)([\w\%\.]*)',r'\2',link)
    original_name = original_name.replace('_',' ').replace('%27',"'")
    original_name = original_name + '::CrAn:' + original_name
    cleaned_name_entity_list.append(original_name)

In [8]:
s = simple_get('https://gravityfalls.fandom.com/wiki/Category:Places')
b = BeautifulSoup(s,'html.parser')
b = b.find_all('div', attrs={'class',"category-page__members"})

loc_links = re.findall(r'href=\"[\w\/\%\.\(\)\&\;\,\-]*\"', str(b))
loc_links = [re.sub(r'(href=\")([\w\/\%\.\(\)\&\;\,\-]*)(\")',r'\2',link) for link in loc_links]
counter = 0
for link in loc_links:
    counter += 1
    original_name = re.sub(r'(\/wiki\/)([\w\%\.]*)',r'\2',link)
    original_name = original_name.replace('_',' ').replace('%27',"'").replace('%26','&')
    original_name = original_name + '::LOC:' + original_name
    cleaned_name_entity_list.append(original_name)
#print(counter)

In [9]:
s = simple_get('https://gravityfalls.fandom.com/wiki/Category:Objects')
b = BeautifulSoup(s,'html.parser')
b = b.select('table td')

random_dict = dict()
counter = 0
count = 1
 
for piece in b:
    individuals = list()
    cleaned_piece = re.sub(r'<td\s?[\w\"\;\=\s\:]*>','',str(piece))
    cleaned_piece = re.sub(r'<\/td>',r'',cleaned_piece)
    cleaned_piece = re.sub(r'(<a[\s\w\=\"\/\%\#\-\,\!\'\:\?\.\(\)\&\;]*>)([\w\s\-\?\,\!\'\:\.\#\&\;]*)(<\/a>)', r'\2',cleaned_piece)
    cleaned_piece = cleaned_piece.replace('<i>','').replace('</i>','').replace('&amp;','and').strip()
    
    if count % 2 != 0:
        title = cleaned_piece
    
    if '|' in cleaned_piece:
        individuals = cleaned_piece.split('|')
        individuals = [w.strip() for w in individuals]
        
    if individuals:
        random_dict[title] = individuals
        
    count +=1
    
tags = ['TEXT','TV','FOOD','VEH','GAME','ELEC','SN','GOV','FT','MS']

for entry in random_dict:
    tag = ''
    end_tag = ''
    list_of_things = random_dict.get(entry)
    
    if 'org' in entry.lower() or 'gov' in entry.lower():
        tag = '::ORG'
    else:
        tag = '::MISC'
        if 'book' in entry .lower() or 'periodical' in entry.lower():
            end_tag = ':TEXT'
        elif 'tv' in entry.lower():
            end_tag = ':TV'
        elif 'food' in entry.lower():
            end_tag = ':FOOD'
        elif 'veh' in entry.lower():
            end_tag = ':VEH'
        elif 'game' in entry.lower():
            end_tag = ':GAME'
        elif 'elec' in entry.lower():
            end_tag = ':ELEC'
        elif 'super' in entry.lower():
            end_tag = ':SN'
        elif 'future' in entry.lower():
            end_tag = ':FT'
        elif 'mys' in entry.lower():
            end_tag = ':MS'
    
    for item in list_of_things:
        counter += 1
        if end_tag not in [':TEXT', ':TV', ':FOOD', ':VEH', ':GAME',
                           ':ELEC', ':SN', ':FT', ':MS']:
            end_tag=':' + item
        #print(item + tag + end_tag)
        cleaned_name_entity_list.append(item + tag + end_tag)
#print(counter)

In [10]:
cleaned_name_entity_list.sort()
completeName = os.path.join(dir, 'named_entities_all/', 'named_entities_gravity_falls.txt') 
f = open(completeName,'w+')
for s in cleaned_name_entity_list:
    f.write("%s\n" % s)
f.close()

In [14]:
ner_file = dir + 'named_entities_all/named_entities_gravity_falls.txt'

with open(ner_file, 'r') as myfile:
    ner = myfile.read()

print(ner)

entities_pieces = []

for n in ner.split('\n'):
    entities_pieces.append(n.split('::'))

final_entities = []

for e in entities_pieces:
    if len(e) > 1 and 'list' not in e[0].lower():
        temp_list = []
        temp_list.append(e[0])
        tags = e[1].split(':')
        for t in tags:
            temp_list.append(t)
        final_entities.append(temp_list)


f = open(dir + 'named_entities_all/gravity_falls_ner.txt', 'w+')
for w in final_entities:
    f.writelines(', '.join(w))
    f.write('\n')
f.close()

.GIFfany::CrAn:.GIFfany
1001 Yuk 'Em Ups::MISC:TEXT
412 Gopher Road::LOC:412 Gopher Road
8 Ball::CHR:8Ball
8 Ball::CrAn:8 Ball
Abandoned church::LOC:Abandoned church
Abandoned mines::LOC:Abandoned mines
Abuelita's house::LOC:Abuelita's house
Abuelita::CHR:Abuelita
Adhesive::MISC:SN
Afficionado Afficionado Magazine::MISC:TEXT
Agent Powers::CHR:APowers
Agent Trigger::CHR:ATrigger
Alex Hirsch (Character)::CHR:A(Character)
America guy::CHR:Aguy
Amorphous Shape::CHR:AShape
Amorphous Shape::CrAn:Amorphous Shape
Andrew "8-Ball" Alcatraz::CHR:SPines
Angry lady::CHR:Alady
Anomalous Phenomena::MISC:TEXT
Aoshima::CrAn:Aoshima
Aquatic restaurant cook::CHR:Acook
Aquatic restaurant::LOC:Aquatic restaurant
Arachnimorphs::CrAn:Arachnimorphs
Arcade::LOC:Arcade
Archibald Corduroy::CrAn:Archibald Corduroy
Avoid Eye Contact Monthly::MISC:TEXT
Axolotl::CrAn:Axolotl
BABBA::ORG:BABBA
Baby Fights::MISC:TV
Baby pterodactyl::CrAn:Baby pterodactyl
Baldy feller::CHR:BIvan
Baron Num Nums High Flyin' Beans::MISC:FO